## Setup

### Reduce Tahoe region greenhouse gas (GHG) emissions and support the measurement of carbon sequestration.  

In [ ]:
# GHG Data
ghgTable = "https://maps.trpa.org/server/rest/services/LTinfo_Climate_Resilience_Dashboard/MapServer/124"

### Track poor air quality, wildfire smoke, and extreme heat trends regionally.

In [ ]:
## getting Purple Air data

# TRPA Air Quality Monitoring data
trpaAirqualityStationLayer = "https://maps.trpa.org/server/rest/services/LTInfo_Monitoring/MapServer/16"
trpaairqualityDailyTable   = "https://maps.trpa.org/server/rest/services/LTInfo_Monitoring/MapServer/17"
trpaairqualityYearlyTable  = "https://maps.trpa.org/server/rest/services/LTInfo_Monitoring/MapServer/46"

In [ ]:
from arcgis.features import FeatureLayer
import pandas as pd
import plotly.express as px

# Purple Air data
purpleAirSensorLayer = "https://maps.trpa.org/server/rest/services/LTinfo_Climate_Resilience_Dashboard/MapServer/143"

# get data as spatially enabled dataframe from feature layer
fl = FeatureLayer(purpleAirSensorLayer)
sdfPurpleAir = fl.query().sdf

# groupby date and get the mean of pm25
sdfPurpleAir = sdfPurpleAir.groupby('date')['mean_pm25'].mean().reset_index()
# creat a plotly express line chart
fig = px.line(sdfPurpleAir, x="date", y="mean_pm25", title='Purple Air PM2.5')
fig.show()

In [ ]:
# color background of plotly by x axis value and gradient fill
fig = px.line(sdfPurpleAir, x="date", y="mean_pm25", title='Purple Air PM2.5', line_shape="spline", render_mode="svg")
fig.update_traces(line_color='blue')
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
fig.show()


### Lake Tahoe water level

In [ ]:
import plotly.express as px
import pandas as pd
import requests
from datetime import datetime, timedelta

# tahoe city site number
site_number = '10337000'

# USGS API URL
url = f'https://waterservices.usgs.gov/nwis/iv/?format=json&sites={site_number}&parameterCd=00065&startDT=2021-01-01&endDT=2021-01-30'

# function to get data from USGS API as dataframe
def get_usgs_data(days):
    site_number = 10337000
    
    # Calculate the start and end dates based on the selected time range
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days)
    
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')
    
    url = f'https://waterservices.usgs.gov/nwis/iv/?format=json&sites={site_number}&parameterCd=00065&startDT={start_date_str}&endDT={end_date_str}'

    response = requests.get(url)
    data = response.json()

    time_series_data = data['value']['timeSeries'][0]['values'][0]['value']

    df = pd.DataFrame(time_series_data)
    df['value'] = pd.to_numeric(df['value'])
    df['dateTime'] = pd.to_datetime(df['dateTime'])
    df['value'] = df['value'] + 6220
    return df

# create plot of lake level
def update_chart(selected_days):
    df = get_usgs_data(selected_days)

    fig = px.line(df, x='dateTime', y='value', title='Lake Tahoe Water Level')
    fig.update_xaxes(title_text='Time')
    fig.update_yaxes(title_text='Water Level (ft)')

    return fig

# create plot of lake level
update_chart(6000)

In [ ]:
# The USGS Site at Tahoe City only has data for the last 5968 days (~16.3 years) October 2007
days = 5968  
df = get_usgs_data(days)
df.info()

### Annual average water temperature, including surface water temperature

In [ ]:
import requests
# get data from TERC service
lakeTempURL = "https://tepfsail50.execute-api.us-west-2.amazonaws.com/v1/report/ns-station-range?rptdate=20240130&rptend=20240202&id=4"
# get data from TERC
response = requests.get(lakeTempURL)

In [ ]:
print(response.json())

### Lake clarity measured by Secchi Depth

In [ ]:
# secchi depth data
secchiDepth    = "https://maps.trpa.org/server/rest/services/LTInfo_Monitoring/MapServer/14"
secchiDepthAll = "https://maps.trpa.org/server/rest/services/LTInfo_Monitoring/MapServer/13"
secchiDepth    = "https://maps.trpa.org/server/rest/services/LTinfo_Climate_Resilience_Dashboard/MapServer/125"


### Total precipitation in water per year, extreme precipitation, and snow as a fraction of annual precipitation
* https://www.ncei.noaa.gov/cdo-web/
* https://www.weather.gov/documentation/services-web-api
* https://www.ncdc.noaa.gov/cdo-web/api/v2/

In [ ]:
import requests
import pandas as pd

# mbindl@trpa.gov API key
api_key = 'LaQrgmyzkUTfbBeUDaUjYiQLLOCXkhYY'

# Define the base URL for the NOAA API
base_url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/'

# Define the endpoint you want to access (e.g., 'stations')
endpoint = 'stations'

# Define the parameters for your request (e.g., dataset, location)
params = {
    'datasetid': 'GHCND',  # Global Historical Climatology Network - Daily (GHCND)
    'locationid': 'FIPS:06',  # FIPS code for California
    'limit': 1000  # Maximum number of results to return
}

# Make the request to the NOAA API
response = requests.get(base_url + endpoint, headers={'token': api_key}, params=params)

# Parse the JSON response
data = response.json()

# The 'results' field of the response contains the list of stations
stations = data['results']

# convert the list of stations to a DataFrame
stations_df = pd.DataFrame(stations)

In [ ]:
stations_df.head()

In [ ]:
# get NOAA data obersavitoins for the last 30 years
# Define the endpoint you want to access (e.g., 'stations')
endpoint = 'data'
# Define the parameters for your request (e.g., dataset, location)
params = {
    'datasetid': 'GHCND',  # Global Historical Climatology Network - Daily (GHCND)
    'locationid': 'FIPS:06',  # FIPS code for California
    'stationid': 'GHCND:USW00023234',  # Station ID for Tahoe City
    'startdate': '1991-01-01',
    'enddate': '2021-01-01',
    'limit': 1000  # Maximum number of results to return
}
# Make the request to the NOAA API
response = requests.get(base_url + endpoint, headers={'token': api_key}, params=params)
# Parse the JSON response
data = response.json()
# The 'results' field of the response contains the list of stations
observations = data['results']
# convert the list of stations to a DataFrame
observations_df = pd.DataFrame(observations)
observations_df.head()

In [ ]:
# get data from NOAA API
# NOAA API URL for Tahoe area
noaaURL = "https://api.weather.gov/gridpoints/MTR/155,80/forecast"
response = requests.get(noaaURL)
data = response.json()
print(data)

### Acres of forest fuels reduction treated for wildfire in high-risk areas, map of prescribed fire treatment projects

In [ ]:
# get EIP indicator as dataframe
import pandas as pd
import plotly.express as px

# EIP Service for Acres Treated
eipForestTreatments = "https://www.laketahoeinfo.org/WebServices/GetReportedEIPIndicatorProjectAccomplishments/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476/19"

# TRCD service ## Doesnt match EIP data ## from 2022 used in map though
trcdLayer = "https://services6.arcgis.com/1KtlSd2mklZMBKaz/ArcGIS/rest/services/Tahoe_Forest_Fuels_Tx_OFFICIAL_Public_View/FeatureServer/0"

# get the data
dfTreatments = pd.read_json(eipForestTreatments)


In [ ]:
dfTreatments.info()

In [ ]:
dfTreatments.to_csv("EIP_ForestHealthTreatments.csv", index=False)

In [ ]:

# get EIP indicator as dataframe
import pandas as pd
import plotly.express as px

# EIP Service for Acres Treated
eipForestTreatments = "https://www.laketahoeinfo.org/WebServices/GetReportedEIPIndicatorProjectAccomplishments/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476/19"

# TRCD service ## Doesnt match EIP data ## from 2022 used in map though
trcdLayer = "https://services6.arcgis.com/1KtlSd2mklZMBKaz/ArcGIS/rest/services/Tahoe_Forest_Fuels_Tx_OFFICIAL_Public_View/FeatureServer/0"

# get the data
dfTreatments = pd.read_json(eipForestTreatments)

# display the data
dfTreatments.head()
# group treatments by year
dfTreatments = dfTreatments.groupby(['IndicatorProjectYear']).sum().reset_index()
dfTreatments.head()
# create plotly figure of forest treatments by year
fig = px.bar(dfTreatments, x='IndicatorProjectYear', y='IndicatorProjectValue', title='Forest Treatments by Year')
fig.update_xaxes(title_text='Year')
fig.update_yaxes(title_text='Acres Treated')
fig.show()

### Tree species diversity and increasing old growth forest 

In [ ]:
from arcgis.features import FeatureLayer
## historical species distribution and old growth data is not comparable to current data

# display Old Growth Forest Acres
oldGrowthLayer = "https://maps.trpa.org/server/rest/services/Vegetation_Late_Seral/FeatureServer/0"

# get the feature layer as a spatially enabled dataframe
# speciesData = FeatureLayer(speciesLayer).query().sdf
oldGrowthData = FeatureLayer(oldGrowthLayer).query().sdf

# summarize the data by Acres
oldGrowthDataSummary = oldGrowthData.groupby('SeralStage').agg({'Acres': 'sum'}).reset_index()



In [ ]:
dfVeg.info()

In [ ]:
import numpy as np
# colors of the veg types
colors = ['#d7d79e','#9ed7c2','#cdf57a','#b4d79e', 
          '#ff0000', '#a5f57a','#00a820','#df73ff', 
          '#3e72b0','#2f3f56', '#a8a800']

# current species distribution Use "TRPA_Veg_Type" to display the current species distribution
speciesLayer = "https://maps.trpa.org/server/rest/services/Vegetation_Type/MapServer/0"

# get the feature layer as a spatially enabled dataframe
dfVeg = FeatureLayer(speciesLayer).query().sdf

# summarize the data by Acres
dfVeg.TRPA_VegType.replace('', np.nan, inplace=True)
dfVegType = dfVeg.groupby("TRPA_VegType")["Acres"].sum().reset_index()

df = dfVegType.rename(columns={'TRPA_VegType':'Vegetation Type'})

df['Vegetation Type'] = df['Vegetation Type'].replace(['Cushion Plant'],'Cushion Plant/Rocky Outcrop')

fig = px.bar(df, y="Vegetation Type", x="Acres", color="Vegetation Type", orientation="h", hover_name="Vegetation Type",
             color_discrete_sequence=colors ,
             title="Vegetation Abundance"
            )

fig.update_traces(hovertemplate='<b>%{x:,.0f}</b> acres<extra></extra>')

# set style variables
template = 'plotly_white'
font     = 'Calibri'

fig.update_layout(font_family=font,
                    template=template,
                    showlegend=False,
                    hovermode="y unified",
                    yaxis={'categoryorder':'total ascending'})
fig.show()

### Probability of fire by low, moderate, & high severity by management zone.

In [ ]:
# spatial analysis for fire severity
import arcpy

# reclass Low Severity flame length raster
with arcpy.EnvManager(scratchWorkspace=r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\ForestHealth_ThresholdUpdate.gdb"):
    out_raster = arcpy.sa.Reclassify(
        in_raster=r"F:\\GIS\\PROJECTS\\ForestHealth_Intiative\\ThresholdUpdate\\Data\\ForestHealth_ThresholdUpdate.gdb\FunctionalFire_ProbLowSeverity_2022_ACCEL_30m_Tahoe",
        reclass_field="Value",
        remap="0 0.600000 0;0.600000 1 1",
        missing_values="DATA"
    )
    out_raster.save(r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\ForestHealth_ThresholdUpdate.gdb\FunctionalFire_Reclass_Low_60thPercentile")

# convert reclassified raster to polygon
arcpy.conversion.RasterToPolygon(
    in_raster=r"Fire Severity\Low Severity Fire",
    out_polygon_features=r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\ForestHealth_ThresholdUpdate.gdb\FunctionalFire_ProbableLowSeverityFire",
    simplify="NO_SIMPLIFY",
    raster_field="Value",
    create_multipart_features="SINGLE_OUTER_PART",
    max_vertices_per_feature=None
)

# identity analysis
arcpy.analysis.Identity(
    in_features=r"Boundaries\Forest Management Zone",
    identity_features="FunctionalFire_ProbableLowSeverityFire",
    out_feature_class=r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\ForestHealth_ThresholdUpdate.gdb\ForestManagementZon_Identity_LowSeverity",
    join_attributes="ALL",
    cluster_tolerance=None,
    relationship="NO_RELATIONSHIPS"
)

# Calculate the area in acres for each feature
arcpy.management.CalculateGeometryAttributes(
    in_features=r"Fire Severity\ForestManagementZon_Identity_LowSeverity",
    geometry_property="Acres AREA",
    length_unit="",
    area_unit="ACRES_US",
    coordinate_system='PROJCS["NAD_1983_UTM_Zone_10N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-123.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]',
    coordinate_format="SAME_AS_INPUT"
)

# export table to database
arcpy.conversion.ExportTable(
    in_table=r"Fire Severity\ForestManagementZon_Identity_LowSeverity",
    out_table=r"F:\GIS\DB_CONNECT\Tabular.sde\SDE.ClimateResilience_ProbabilityLowSeverityFire_by_ForestManagmentZone",
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
    sort_field=None
)

# reclass High Severity flame length raster
with arcpy.EnvManager(scratchWorkspace=r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\ForestHealth_ThresholdUpdate.gdb"):
    out_raster = arcpy.sa.Reclassify(
        in_raster=r"F:\\GIS\\PROJECTS\\ForestHealth_Intiative\\ThresholdUpdate\\Data\\ForestHealth_ThresholdUpdate.gdb\FunctionalFire_ProbHighSeverity_2022_ACCEL_30m_Tahoe",
        reclass_field="Value",
        remap="0 0.600000 0;0.600000 1 1",
        missing_values="DATA"
    )
    out_raster.save(r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\ForestHealth_ThresholdUpdate.gdb\FunctionalFire_Reclass_High_60thPercentile")

# convert reclassified raster to polygon
arcpy.conversion.RasterToPolygon(
    in_raster=r"Fire Severity\High Severity Fire",
    out_polygon_features=r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\ForestHealth_ThresholdUpdate.gdb\FunctionalFire_ProbableHighSeverityFire",
    simplify="NO_SIMPLIFY",
    raster_field="Value",
    create_multipart_features="SINGLE_OUTER_PART",
    max_vertices_per_feature=None
)

# identity analysis
arcpy.analysis.Identity(
    in_features=r"Boundaries\Forest Management Zone",
    identity_features="FunctionalFire_ProbableHighSeverityFire",
    out_feature_class=r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\ForestHealth_ThresholdUpdate.gdb\ForestManagementZon_Identity_HighSeverity",
    join_attributes="ALL",
    cluster_tolerance=None,
    relationship="NO_RELATIONSHIPS"
)

# Calculate the area in acres for each feature
arcpy.management.CalculateGeometryAttributes(
    in_features=r"Fire Severity\ForestManagementZon_Identity_HighSeverity",
    geometry_property="Acres AREA",
    length_unit="",
    area_unit="ACRES_US",
    coordinate_system='PROJCS["NAD_1983_UTM_Zone_10N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-123.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]',
    coordinate_format="SAME_AS_INPUT"
)

# export table to database
arcpy.conversion.ExportTable(
    in_table=r"Fire Severity\ForestManagementZon_Identity_HighSeverity",
    out_table=r"F:\GIS\DB_CONNECT\Tabular.sde\SDE.ClimateResilience_ProbabilityHighSeverityFire_by_ForestManagmentZone",
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
    sort_field=None
)


In [ ]:
import pandas as pd
# import packages
from arcgis.features import FeatureLayer
import pandas as pd
import numpy as np
import plotly.express as px

# Define the service URL
highseverityURL = "https://maps.trpa.org/server/rest/services/LTinfo_Climate_Resilience_Dashboard/MapServer/129"
lowseverityURL  = "https://maps.trpa.org/server/rest/services/LTinfo_Climate_Resilience_Dashboard/MapServer/130"

# Create a FeatureLayer object
feature_layer = FeatureLayer(highseverityURL)

# Query the feature layer and convert the result to a Spatially Enabled DataFrame
sdf_highseverity = feature_layer.query().sdf

sdf_highseverity.head()
# summarize the area of high and low severity fire by name and gridcode
highseverity_summary = sdf_highseverity.groupby(['Name', 'gridcode'])['Acres'].sum().reset_index()

# reclassify values of gridcode in new field called 'Severity' with 1 = ">60% chance of high severity fire" and 0 = "<60% chance of high severity fire"
highseverity_summary['Severity'] = np.where(highseverity_summary['gridcode'] == 1, ">60% chance of high severity fire", "<60% chance of high severity fire")

# Plot using Plotly Express to create a stacked bar chart of Severity by Name with the bar chart being 100% stacked
fig = px.histogram(highseverity_summary, x='Name', y='Acres', color='Severity', title='High Severity Fire by Forest Management Zone', barnorm='percent',barmode='stack')
# fig = px.bar(highseverity_summary, x='Name', y='Acres', color='Severity', title='High Severity Fire by Forest Management Zone', barmode='stack')
fig.show()


In [ ]:
lowseverityURL  = "https://maps.trpa.org/server/rest/services/LTinfo_Climate_Resilience_Dashboard/MapServer/130"

# Create a FeatureLayer object
feature_layer = FeatureLayer(lowseverityURL)

# Query the feature layer and convert the result to a Spatially Enabled DataFrame
sdf_lowseverity = feature_layer.query().sdf

sdf_lowseverity.head()
# summarize the area of high and low severity fire by name and gridcode
lowseverity_summary = sdf_lowseverity.groupby(['Name', 'gridcode'])['Acres'].sum().reset_index()

# reclassify values of gridcode in new field called 'Severity' 
lowseverity_summary['Severity'] = np.where(lowseverity_summary['gridcode'] == 1, ">60% chance of low severity fire", "<60% chance of low severity fire")

# Plot using Plotly Express to create a stacked bar chart of Severity by Name with the bar chart being 100% stacked
fig = px.histogram(lowseverity_summary, x='Name', y='Acres', color='Severity', title='Low Severity Fire by Forest Management Zone', barnorm='percent',barmode='stack')
# fig = px.bar(highseverity_summary, x='Name', y='Acres', color='Severity', title='High Severity Fire by Forest Management Zone', barmode='stack')
fig.show()

In [ ]:
# Data for Moderate Severity Fire does not exist in the service....USFS data is needed to complete the analysis, but they didnt provide it in the data package

# we could get at this by doing the math's between low and high severity fire, but that would be a bit of a stretch


### Acres treated for aquatic invasive species. *We could gain additional clarity regarding this indicator.

In [ ]:
# get EIP indicator as dataframe
import pandas as pd
import plotly.express as px
# EIP Service for Acres Treated
eipInvasive = "https://www.laketahoeinfo.org/WebServices/GetReportedEIPIndicatorProjectAccomplishments/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476/15"

df = pd.read_json(eipInvasive)

sum_df = df.groupby('IndicatorProjectYear')['IndicatorProjectValue'].cumsum()

display(sum_df)


### Acres of restored high-quality wetlands and meadows helping to store flood waters. *We are expecting additional clarity regarding this indicator.  

In [ ]:
# get EIP indicator as dataframe
import pandas as pd
import plotly.express as px

# EIP Service for SEZ Restored or Enhanced
eipSEZRestored = "https://www.laketahoeinfo.org/WebServices/GetReportedEIPIndicatorProjectAccomplishments/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476/9"

# GIS of SEZ Enchanced
trpaSEZEnhancedRestored = "https://maps.trpa.org/server/rest/services/LTinfo_Climate_Resilience_Dashboard/MapServer/126"
# GIS of SEZ Restored
trpaSEZRestored = "https://maps.trpa.org/server/rest/services/LTinfo_Climate_Resilience_Dashboard/MapServer/127"
# get the data
df = pd.read_json(eipSEZRestored)


### Increased number of parcels with Stormwater Best Management Practices (BMPs) improvements

In [ ]:
# get BMP data as dataframe
import pandas as pd
import plotly.express as px
from arcgis.features import FeatureLayer

# # EIP Service for BMPs installed
# bmpsInstalled = "https://www.laketahoeinfo.org/WebServices/GetReportedEIPIndicatorProjectAccomplishments/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476/4"

# BMP map service from BMP database
bmpsLayer = "https://maps.trpa.org/server/rest/services/LTinfo_Climate_Resilience_Dashboard/MapServer/121"

# get the data
feature_layer = FeatureLayer(bmpsLayer)

# Query the feature layer and convert the result to a Spatially Enabled DataFrame
sdf_bmps = feature_layer.query().sdf

# total BMPs installed
total_bmps = sdf_bmps['OBJECTID'].count()

# filter total BMPs CertificateIssued = 1
sdf_bmps_cert = sdf_bmps[sdf_bmps['CertificateIssued'] == 1]

# total BMPs installed
total_bmps_cert = sdf_bmps_cert['OBJECTID'].count()

# create Year column
sdf_bmps_cert['Year'] = pd.DatetimeIndex(sdf_bmps_cert['CertDate']).year

# total BMPs installed per year
total_bmps_cert_year = sdf_bmps_cert.groupby('Year')['OBJECTID'].count().reset_index()

# create plotly figure of BMPs installed per year
fig = px.bar(total_bmps_cert_year, x='Year', y='OBJECTID', title='BMPs Installed per Year')
fig.update_xaxes(title_text='Year')
fig.update_yaxes(title_text='BMPs Installed')
fig.show()

In [ ]:
# cumulutive sum of BMPs installed per year
total_bmps_cert_year['Cumulative BMPs Installed'] = total_bmps_cert_year['OBJECTID'].cumsum()
# create plotly figure of BMPs installed per year
fig = px.bar(total_bmps_cert_year, x='Year', y='Cumulative BMPs Installed', title='BMPs Installed')
fig.update_xaxes(title_text='Year')
fig.update_yaxes(title_text='Cumulative BMPs Installed')
fig.show()


In [ ]:
# BMP map service from BMP database
bmpsLayer = "https://maps.trpa.org/server/rest/services/LTinfo_Climate_Resilience_Dashboard/MapServer/121"
# get the data
feature_layer = FeatureLayer(bmpsLayer)

# Query the feature layer and convert the result to a Spatially Enabled DataFrame
sdf_bmps = feature_layer.query().sdf

# filter total BMPs CertificateIssued = 1
sdf_bmps_cert = sdf_bmps[sdf_bmps['CertificateIssued'] == 1]

# total developed parcels, use .loc to get EXISTING_LANDUSE does not equal "Vacant" or "Open Space"
parcelsDeveloped = sdf_bmps.loc[~sdf_bmps['EXISTING_LANDUSE'].isin(['Vacant', 'Open Space'])]
parcelsDeveloped = parcelsDeveloped['OBJECTID'].count()

# total bmps certified by year compared to total developed parcels
# create Year column
sdf_bmps_cert['Year'] = pd.DatetimeIndex(sdf_bmps_cert['CertDate']).year
bmpsCertByYear = sdf_bmps_cert.groupby('Year')['OBJECTID'].count().reset_index()
bmpsCertByYear['Cumulative BMPs Installed'] = bmpsCertByYear['OBJECTID'].cumsum()
bmpsCertByYear['Developed Parcels'] = parcelsDeveloped
bmpsCertByYear['BMPs per Developed Parcel'] = bmpsCertByYear['Cumulative BMPs Installed'] / bmpsCertByYear['Developed Parcels']
bmpsCertByYear['BMPs per Developed Parcel'] = bmpsCertByYear['BMPs per Developed Parcel'].round(2)

# create plotly figure of BMPs installed per year
fig = px.bar(bmpsCertByYear, x='Year', y='BMPs per Developed Parcel', title='BMPs Installed per Developed Parcel')
fig.update_xaxes(title_text='Year')
fig.update_yaxes(title_text='% BMPs Installed per Developed Parcel')
fig.show()

In [ ]:
# create staked bar chart of BMPs installed per year compared to total developed parcels per year but subtracting the BMPs installed from the total developed parcels
bmpsCertByYear['Developed Parcels Without a BMP'] = bmpsCertByYear['Developed Parcels'] - bmpsCertByYear['Cumulative BMPs Installed']

bmpsCertByYear

In [ ]:
# create staked bar chart of BMPs installed per year compared to total developed parcels per year but subtracting the BMPs installed from the total developed parcels
fig = px.bar(bmpsCertByYear, x='Year', y=['Cumulative BMPs Installed', 'Developed Parcels Without a BMP'], title='BMPs Installed per Developed Parcel')
fig.update_xaxes(title_text='Year')
fig.update_yaxes(title_text='BMPs Installed per Developed Parcel')
fig.show()


### Increase in square feet of urban development treated by areawide stormwater infrastructure within key watersheds.   

In [ ]:
## reqeusted "Year of Completion" data for each area wide storm water treatment from Shay. TBD if this is available

# area wide storm water treamtent layer # we can't do this analysis without the year of completion data
areawideLayer  = "https://maps.trpa.org/server/rest/services/LTinfo_Climate_Resilience_Dashboard/MapServer/120"

# developed area layers
impervious2010Layer   = "https://maps.trpa.org/server/rest/services/Impervious_Surface_2010/MapServer"
impervious2019Layer   = "https://maps.trpa.org/server/rest/services/Impervious_Surface_2019/MapServer"
imperviousChangeLayer = "https://maps.trpa.org/server/rest/services/Impervious_Surface_Change_2010_to_2019/MapServer"

In [ ]:
arcpy.analysis.Identity(
    in_features=r"F:\GIS\DB_CONNECT\Vector.sde\SDE.Impervious\SDE.Impervious_2019",
    identity_features=r"F:\GIS\DB_CONNECT\Vector.sde\SDE.EIP\SDE.Existing_Drainage_Areas",
    out_feature_class=r"C:\GIS\Scratch.gdb\Impervious_2019_Identity",
    join_attributes="ALL",
    cluster_tolerance=None,
    relationship="NO_RELATIONSHIPS"
)

In [ ]:
arcpy.management.CalculateGeometryAttributes(
    in_features="Impervious_2019_Identity",
    geometry_property="Acres AREA",
    length_unit="",
    area_unit="ACRES_US",
    coordinate_system='PROJCS["NAD_1983_UTM_Zone_10N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-123.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]',
    coordinate_format="SAME_AS_INPUT"
)

arcpy.conversion.ExportTable(
    in_table="Impervious_2019_Identity",
    out_table=r"F:\GIS\DB_CONNECT\Tabular.sde\SDE.ClimateResilience_Impervious_Identify_Areawide_ByYear",
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping='FID_Impervious_2019 "FID_Impervious_2019" true true false 4 Long 0 0,First,#,Impervious_2019_Identity,FID_Impervious_2019,-1,-1;Feature "Feature" true true false 8 Text 0 0,First,#,Impervious_2019_Identity,Feature,0,7;Surface "Surface" true true false 4 Text 0 0,First,#,Impervious_2019_Identity,Surface,0,3;FID_Existing_Drainage_Areas "FID_Existing_Drainage_Areas" true true false 4 Long 0 0,First,#,Impervious_2019_Identity,FID_Existing_Drainage_Areas,-1,-1;Drainage_Area_Name "Name" true true false 100 Text 0 0,First,#,Impervious_2019_Identity,Drainage_Area_Name,0,99;Status "Status" true true false 50 Text 0 0,First,#,Impervious_2019_Identity,Status,0,49;Year_Completed "Year_Completed" true true false 4 Text 0 0,First,#,Impervious_2019_Identity,Year_Completed,0,3;Acres "Acres" true true false 8 Double 0 0,First,#,Impervious_2019_Identity,Acres,-1,-1',
    sort_field=None
)

In [ ]:
import pandas as pd
import plotly.express as px
from arcgis.features import FeatureLayer

# REST service for impervious area wide
imperviousAreaWide = "https://maps.trpa.org/server/rest/services/LTinfo_Climate_Resilience_Dashboard/MapServer/140"

# Create a FeatureLayer object
feature_layer = FeatureLayer(imperviousAreaWide)
# Query the feature layer and convert the result to a Spatially Enabled DataFrame
sdf_impervious = feature_layer.query().sdf

# summarize total area of Surface = Hard Surface
sdf_impervious_hard = sdf_impervious[sdf_impervious['Surface'] == 'Hard']

# summarize the area of hard surface covered by status = completed or active
sdf_impervious_hard_summary = sdf_impervious_hard.groupby(['Status', 'Year_Completed'])['Acres'].sum().reset_index()

sdf_impervious_hard_summary



In [ ]:
# filter out Status is not ''
sdf_impervious_hard_summary = sdf_impervious_hard_summary[sdf_impervious_hard_summary['Status'] != '']
# group status active and constructed to completed
sdf_impervious_hard_summary['Status'] = sdf_impervious_hard_summary['Status'].replace(['Active', 'Constructed'], 'Completed')

# add acres in cumulative sum
sdf_impervious_hard_summary['Cumulative Acres'] = sdf_impervious_hard_summary.groupby('Status')['Acres'].cumsum()

# ploty figure of impervious area wide by year
fig = px.bar(sdf_impervious_hard_summary, x='Year_Completed', y='Cumulative Acres', color='Status', title='Impervious Area Wide by Year')
fig.update_xaxes(title_text='Year')
fig.update_yaxes(title_text='Acres of Hard Surface Covered by Yera')
fig.show()

In [ ]:
# get total acreas of surface == hard

total_acres = sdf_impervious_hard['Acres'].sum()
total_acres

# create cumulative sum of acres of status - completed
sdf_impervious_hard_summary['Cumulative Acres'] = sdf_impervious_hard_summary['Acres'].cumsum()

# subtract area covereed by cumulatve sum from total acres

sdf_impervious_hard_summary['Acres Remaining'] = total_acres - sdf_impervious_hard_summary['Cumulative Acres']

sdf_impervious_hard_summary

In [ ]:
# ploty figure of impervious area wide by year
fig = px.bar(sdf_impervious_hard_summary, x='Year_Completed', y=['Cumulative Acres', 'Acres Remaining'], title='Impervious Area Wide by Year')
fig.update_xaxes(title_text='Year')
fig.update_yaxes(title_text='Acres of Hard Surface Covered by Yera')
fig.show()

### Total number of housing units in town centers and share of affordable housing in Town Centers

In [ ]:
# town center layer
towncenterLayer = "https://maps.trpa.org/server/rest/services/LocalPlan/MapServer/2"
# parcel development history layer for 2012, and 2018-2022 # 2021 and 2023 coming soon. other years are unknown
parcelDevelopmentHistoryLayer = "https://maps.trpa.org/server/rest/services/LTinfo_Climate_Resilience_Dashboard/MapServer/17"
# deed restricted housing layer
deedRestrictedHousingLayer = "https://maps.trpa.org/server/rest/services/LTinfo_Climate_Resilience_Dashboard/MapServer/20"

In [ ]:
# get the feature layer as a spatially enabled dataframe


### Change in share of homes with electric or solar energy fuel compared to oil/gas over time

In [ ]:
# kathleen did this. need to get the data in our database and web service.

### Number of deed-restricted affordable, moderate, and achievable units

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

# deed restriction service
deedRestrictionService = "https://www.laketahoeinfo.org/WebServices/GetDeedRestrictedParcels/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476"

# read in deed restricted parcels
dfDeed = pd.read_json(deedRestrictionService)

# filter out deed restrictions that are not affordable housing
dfDeed = dfDeed.loc[dfDeed['DeedRestrictionType'].isin(['Affordable Housing', 'Achievable Housing', 'Moderate Income Housing'])]

# create year column
dfDeed['Year'] = dfDeed['RecordingDate'].str[-4:]

# group by type and year
df = dfDeed.groupby(['DeedRestrictionType', 'Year']).size().reset_index(name='Total')

# sort by year
df.sort_values('Year', inplace=True)

# rename columns
df = df.rename(columns={'DeedRestrictionType': 'Type', 'Year': 'Year', 'Total': 'Count'})

# Create a DataFrame with all possible combinations of 'Type' and 'Year'
df_all = pd.DataFrame({
    'Type': np.repeat(df['Type'].unique(), df['Year'].nunique()),
    'Year': df['Year'].unique().tolist() * df['Type'].nunique()
})

# Merge the new DataFrame with the original one to fill in the gaps of years for each type with NaN values
df = pd.merge(df_all, df, on=['Type', 'Year'], how='left')

# Replace NaN values in 'Count' with 0
df['Count'] = df['Count'].fillna(0)

# Ensure 'Count' is of integer type
df['Count'] = df['Count'].astype(int)

# Recalculate 'Cumulative Count' as the cumulative sum of 'Count' within each 'Type' and 'Year'
df['Cumulative Count'] = df.sort_values('Year').groupby('Type')['Count'].cumsum()

# create cumuluative total of deed restricted parcels by type
fig = px.line(df, x="Year", y="Cumulative Count", color="Type", title="Deed Restricted Parcels")
fig.show()


### Percent of renewable energy as a share of total energy used

In [ ]:
# kathleen did this. 

### Total transit ridership by transit systems
* https://www.laketahoeinfo.org/Indicator/Detail/46/Overview
* this doesn't work yet. Talking to ESA to get web service stood up.

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from arcgis.features import FeatureLayer

# Data not avaialble from LTinfo yet. Shannon is working on a service for this. I will get the data from Kira as a placeholder for now.
# indicator data saved from Kira's email
eipTransit = "https://maps.trpa.org/server/rest/services/LTinfo_Climate_Resilience_Dashboard/MapServer/131"

# read in spatial enabled dataframe
dfTransit = FeatureLayer(eipTransit).query().sdf

# drop ObjectID
dfTransit = dfTransit.drop(columns=['OBJECTID'])
# stack data by month
dfTransit = dfTransit.melt(id_vars=['MONTH'], var_name='Name', value_name='Ridership')

# create Year field from last two characters of month but add 20 prefix
dfTransit['Year'] = '20' + dfTransit['MONTH'].str[-2:]
# strip the last three characters from month
dfTransit['Month'] = dfTransit['MONTH'].str[:-3]
# drop MONTH
dfTransit = dfTransit.drop(columns=['MONTH'])
# make the values in Month the real names of the months
dfTransit['Month'] = dfTransit['Month'].replace(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], 
                                                ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'])


# create a Date type field of Month and Year
dfTransit['Date'] = dfTransit['Month'] + ' ' + dfTransit['Year']
# convert Date to datetime
dfTransit['Date'] = pd.to_datetime(dfTransit['Date'], format='%B %Y')
dfTransit = dfTransit.sort_values('Date')


In [ ]:
dfTransit



In [ ]:
# create a plotly figure of transit ridership by date 
fig = px.line(dfTransit, x="Date", y="Ridership", color="Name", title="Transit Ridership")
fig.show()

### Daily per capita Vehicles Miles Traveled (VMT) and progress towards VMT target. The RTP (2021) identifies a Daily per capita VMT target set at a 6.8 percent reduction from 2018 levels by 2045 (2018 per capita daily VMT is 12.48, goal is 11.63).

In [ ]:
## new data from Josh coming 2/6/2024

### Coverage of electric bus routes and alternative fuel, such as EV charging for vehicles and bicycles
* Primary source: https://www.plugshare.com/ requested access to their API
* Secondary (if needed to cross-check): https://afdc.energy.gov/stations/#/find/nearest

* current in-house data: https://maps.trpa.org/server/rest/services/LTinfo_Climate_Resilience_Dashboard/MapServer/0

In [ ]:
# no real info on this right now. Will checking with Kira on this.

### Baseline mode share and weekday or seasonal variation. The Tahoe RTP (2021) includes the following Non-Auto Mode Share Target: Improve average non-auto mode share calculated from the two most recent TRPA travel survey results; the current performance on target at 24.5% (2018-20 average) up from 18% in 2014-16.

In [ ]:
## new data from Josh coming 2/6/2024

### Transportation access in priority communities. The Tahoe RTP (2021) includes a target to increase access to each mode for Priority communities to 100% by 2014.

In [ ]:
# will check in with Kira on this again

### Increased lane miles of low-stress bicycle facilities (both bicycle and pedestrian facilities that are considered comfortable enough for all users and abilities, and implicitly measures active transportation network connectivity)

In [ ]:
# import packages
from arcgis.features import FeatureLayer
import pandas as pd
import numpy as np
import plotly.express as px

# Define the service URL
bikelaneService = "https://maps.trpa.org/server/rest/services/Transportation/MapServer/3"

# Create a FeatureLayer object
feature_layer = FeatureLayer(bikelaneService)

# Query the feature layer and convert the result to a Spatially Enabled DataFrame
sdf_bikelane = feature_layer.query().sdf

# recalc miles field from shape length
sdf_bikelane.MILES = sdf_bikelane["Shape.STLength()"]/ 1609.34

# filter for CLASS = 1 2 or 3
filtered_sdf_bikelane = sdf_bikelane[sdf_bikelane['CLASS'].isin(['1', '2', '3'])]

# fix bad values
filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'] = filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'].replace(['before 2010', ' before 2010'], '2010')
filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'] = filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'].replace(['before 2006','Before 2006','BEFORE 2006'], '2006')
filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'] = filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'].replace([' 2014'], '2014')
filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'] = filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'].replace(['2007 (1A) 2008 (1B)'], '2008')

# drop rows with <NA> values
filtered_sdf_bikelane = filtered_sdf_bikelane.dropna(subset=['YR_OF_CONS'])
# drop rows with 'i dont know' or 'UNKNOWN' values
filtered_sdf_bikelane = filtered_sdf_bikelane[~filtered_sdf_bikelane['YR_OF_CONS'].isin(['i dont know', 'UNKNOWN'])]

# rename columns
df = filtered_sdf_bikelane.rename(columns={'CLASS': 'Class', 'YR_OF_CONS': 'Year', 'MILES': 'Miles'})


# Create a DataFrame with all possible combinations of 'Type' and 'Year'
df_all = pd.DataFrame({
    'Class': np.repeat(df['Class'].unique(), df['Year'].nunique()),
    'Year': df['Year'].unique().tolist() * df['Class'].nunique()
})

# Merge the new DataFrame with the original one to fill in the gaps of years for each type with NaN values
df = pd.merge(df_all, df, on=['Class', 'Year'], how='left')

# add 2005 to the Year field for Class 1 2, and 3
dict = {'Class':['1', '2', '3'], 
        'Year':['2005', '2005', '2005'], 
        'Miles':[0, 0, 0] 
       } 
  
df2 = pd.DataFrame(dict) 
  
df = pd.concat([df, df2], ignore_index = True) 
# cast Year as integer
df['Year'] = df['Year'].astype(int)
# sort by year and miles
df.sort_values(['Year', 'Miles'], inplace=True)

# Replace NaN values in 'MILES' with 0
df['Miles'] = df['Miles'].fillna(0)

# Recalculate 'Cumulative Count' as the cumulative sum of 'Count' within each 'Type' and 'Year'
df['Cumulative Count'] = df.sort_values('Year').groupby(['Year', 'Class'])['Miles'].cumsum()

# get rid of all columns except for Year, Class, and Cumulative Count
df = df[['Year', 'Class', 'Cumulative Count']]

df

In [ ]:
# get rid of all columns except for Year, Class, and Cumulative Count
df = df[['Year', 'Class', 'Cumulative Miles']]
# get all rows where Year = 2006
df_2006 = df[df['Year'] == 2006]
# display all rows
pd.set_option('display.max_rows', None)
df_2006

In [ ]:
print(years)

In [ ]:
# import packages
from arcgis.features import FeatureLayer
import pandas as pd
import numpy as np
import plotly.express as px

# Define the service URL
bikelaneService = "https://maps.trpa.org/server/rest/services/Transportation/MapServer/3"

# Create a FeatureLayer object
feature_layer = FeatureLayer(bikelaneService)

# Query the feature layer and convert the result to a Spatially Enabled DataFrame
sdf_bikelane = feature_layer.query().sdf

# recalc miles field from shape length
sdf_bikelane.MILES = sdf_bikelane["Shape.STLength()"]/ 1609.34

# filter for CLASS = 1 2 or 3
filtered_sdf_bikelane = sdf_bikelane[sdf_bikelane['CLASS'].isin(['1', '2', '3'])]

# fix bad values
filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'] = filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'].replace(['before 2010', ' before 2010'], '2010')
filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'] = filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'].replace(['before 2006','Before 2006','BEFORE 2006'], '2006')
filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'] = filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'].replace([' 2014'], '2014')
filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'] = filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'].replace(['2007 (1A) 2008 (1B)'], '2008')

# drop rows with <NA> values
filtered_sdf_bikelane = filtered_sdf_bikelane.dropna(subset=['YR_OF_CONS'])
# drop rows with 'i dont know' or 'UNKNOWN' values
filtered_sdf_bikelane = filtered_sdf_bikelane[~filtered_sdf_bikelane['YR_OF_CONS'].isin(['i dont know', 'UNKNOWN'])]

# rename columns
df = filtered_sdf_bikelane.rename(columns={'CLASS': 'Class', 'YR_OF_CONS': 'Year', 'MILES': 'Miles'})

# Create a DataFrame with all possible combinations of 'Type' and 'Year'
df_all = pd.DataFrame({
    'Class': np.repeat(df['Class'].unique(), df['Year'].nunique()),
    'Year': df['Year'].unique().tolist() * df['Class'].nunique()
})

# Merge the new DataFrame with the original one to fill in the gaps of years for each type with NaN values
df = pd.merge(df_all, df, on=['Class', 'Year'], how='left')

# add 2005 to the Year field for Class 1 2, and 3
dict = {'Class':['1', '2', '3'], 
        'Year':['2005', '2005', '2005'], 
        'Miles':[0, 0, 0] 
       } 
  
df2 = pd.DataFrame(dict) 

# bring in 2005 data  
df = pd.concat([df, df2], ignore_index = True) 

# cast Year as integer
df['Year'] = df['Year'].astype(int)

# sort by year and miles
df.sort_values(['Year', 'Miles'], inplace=True)

# Replace NaN values in 'MILES' with 0
df['Miles'] = df['Miles'].fillna(0)

# 
df =df.groupby(['Year', 'Class'])['Miles'].sum().reset_index()


df



In [ ]:
# import packages
from arcgis.features import FeatureLayer
import pandas as pd
import numpy as np
import plotly.express as px

# Define the service URL
bikelaneService = "https://maps.trpa.org/server/rest/services/Transportation/MapServer/3"

# Create a FeatureLayer object
feature_layer = FeatureLayer(bikelaneService)

# Query the feature layer and convert the result to a Spatially Enabled DataFrame
sdf_bikelane = feature_layer.query().sdf

# recalc miles field from shape length
sdf_bikelane.MILES = sdf_bikelane["Shape.STLength()"]/ 1609.34

# filter for CLASS = 1 2 or 3
filtered_sdf_bikelane = sdf_bikelane[sdf_bikelane['CLASS'].isin(['1', '2', '3'])]

# fix bad values
filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'] = filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'].replace(['before 2010', ' before 2010'], '2010')
filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'] = filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'].replace(['before 2006','Before 2006','BEFORE 2006'], '2006')
filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'] = filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'].replace([' 2014'], '2014')
filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'] = filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'].replace(['2007 (1A) 2008 (1B)'], '2008')

# drop rows with <NA> values
filtered_sdf_bikelane = filtered_sdf_bikelane.dropna(subset=['YR_OF_CONS'])
# drop rows with 'i dont know' or 'UNKNOWN' values
filtered_sdf_bikelane = filtered_sdf_bikelane[~filtered_sdf_bikelane['YR_OF_CONS'].isin(['i dont know', 'UNKNOWN'])]

# rename columns
df = filtered_sdf_bikelane.rename(columns={'CLASS': 'Class', 'YR_OF_CONS': 'Year', 'MILES': 'Miles'})

# Create a DataFrame with all possible combinations of 'Type' and 'Year'
df_all = pd.DataFrame({
    'Class': np.repeat(df['Class'].unique(), df['Year'].nunique()),
    'Year': df['Year'].unique().tolist() * df['Class'].nunique()
})

# Merge the new DataFrame with the original one to fill in the gaps of years for each type with NaN values
df = pd.merge(df_all, df, on=['Class', 'Year'], how='left')

# add 2005 to the Year field for Class 1 2, and 3
dict = {'Class':['1', '2', '3'], 
        'Year':['2005', '2005', '2005'], 
        'Miles':[0, 0, 0] 
       } 
  
df2 = pd.DataFrame(dict) 

# bring in 2005 data  
df = pd.concat([df, df2], ignore_index = True) 

# cast Year as integer
df['Year'] = df['Year'].astype(int)

# sort by year and miles
df.sort_values(['Year', 'Miles'], inplace=True)

# Replace NaN values in 'MILES' with 0
df['Miles'] = df['Miles'].fillna(0)

# create grouped dataframe
df = df.groupby(['Year', 'Class'])['Miles'].sum().reset_index()

# Recalculate 'Cumulative Count' as the cumulative sum of 'Count' within each 'Type' and 'Year'
df['Cumulative Miles'] = df.sort_values('Year').groupby('Class')['Miles'].cumsum()

# create cumuluative total of miles of bike lanes by Class
fig = px.line(df, x="Year", y="Cumulative Miles", color="Class", title="Miles of Bike Lane by Type")
fig.show()


### Median Household Income (MHI) by community area (to be determined) and disaggregated by remote and non-remote workers

In [ ]:
# census table
censusTable = "https://maps.trpa.org/server/rest/services/LTinfo_Climate_Resilience_Dashboard/MapServer/128"

### Housing costs (median home sales price and rental rates, by jurisdiction); include cost per unit and cost per square foot

In [ ]:
# Sean is getting COSTAR and Property Radar data for this

### Housing tenure (rented full-time, owner-occupied, second home), disaggregated by race, ethnicity, and age

In [ ]:
# census data
censusTable = "https://maps.trpa.org/server/rest/services/LTinfo_Climate_Resilience_Dashboard/MapServer/128"

### Percent of workers who commute into the basin, origin demographics, distance travelled, difference in travel time by mode

### Transient Occupancy Tax revenue and changes over time

In [ ]:
## new data from Josh coming 2/6/2024

### Consistent employment and median wages by sector and overall 

### Access to recreation sites, fresh food, and healthcare for zero-vehicle households

In [ ]:
# checking wiht Kira on this. Will make a map of what we have.

### Firewise communities in the Tahoe basin, coolling centers/heating centers, resources, and emergency infrastructure (medical centers with supplies, fire response)

In [ ]:
# map made.

### Population disaggregated by race and ethnicity, age groups

In [ ]:
# census data
censusTable = "https://maps.trpa.org/server/rest/services/LTinfo_Climate_Resilience_Dashboard/MapServer/128"

### Number/share of households with access and functional needs (These can be referred to as vulnerable populations including populations such as persons with disabilities, older adults, children, limited English proficiency, and transportation disadvantages)

In [ ]:
# census data
censusTable = "https://maps.trpa.org/server/rest/services/LTinfo_Climate_Resilience_Dashboard/MapServer/128"